# 数据加载、存储与文件格式

输入输出通常可以划分为几类：
- 读取文本文件和其他更高效的磁盘储存格式
- 加载数据库中的数据
- 利用Web API操作网络资源

## 读取文本格式的数据

![表6-1 pandas中的解析函数](Img/7178691-958f849e6067b19b.png)

将文本数据读取为DataFrame常用的函数可以分为以下几类：
- 索引：将一个或多个列当做返回的DataFrame处理，以及是否从文件、用户获取列名；
- 类型推断和数据转换：包括用户定义值的转换和自定义的缺失值标记列表等；
- 日期解析：包括组合功能，比如将分散在多个列的日期时间信息组合成结果中的单个列；
- 迭代：支持对大文件进行逐块迭代；
- 不规整数据问题：跳过一些行、页脚、注释或其他一些不重要的东西（比如由成千上万个逗号隔开的数值数据）。


### 读取文本数据

`read_csv()`和`read_table()`函数的参数：

![](Img/7178691-082daf4a00ed9494.png)
![](Img/7178691-597327ade3e94c7a.png)

In [1]:
# 看一份数据，Linux shell 命令
!cat Examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('Examples\ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_table('Examples\ex1.csv',sep=',') # 指定分隔符

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# 没有 head
!cat Examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [8]:
pd.read_csv('Examples\ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
pd.read_csv('Examples\ex2.csv',names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
# 指定索引
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('Examples\ex2.csv',names=names,index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [11]:
# 多个列做成层次化索引
!cat Examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [12]:
parsed = pd.read_csv('Examples\csv_mindex.csv',
                    index_col=['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [13]:
# 不能用固定分隔符分隔字段的表格
list(open('Examples\ex3.txt'))
# 被数量不同的空白字符分隔

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [15]:
# 传递正则表达式作为分隔符
result = pd.read_table('Examples/ex3.txt',sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [17]:
# 跳过第 几 行
!cat Examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [16]:
pd.read_csv('Examples/ex4.csv',skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [18]:
# 缺失值
!cat Examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [20]:
result = pd.read_csv('Examples\ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [21]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [23]:
result = pd.read_csv('Examples\ex5.csv',na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [24]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('Examples\ex5.csv',na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 逐块读取文本文件

In [25]:
# 设置pandas显示更紧凑
pd.options.display.max_rows = 10

In [26]:
result = pd.read_csv('Examples\ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [27]:
# 读取指定行
pd.read_csv('Examples\ex6.csv',nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [34]:
# 逐块读取：chunksize(行数)
chunker = pd.read_csv('Examples\ex6.csv',chunksize=1000)
chunker # 返回 TextParser 对象

In [37]:
chunker = pd.read_csv('Examples\ex6.csv',chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(),fill_value=0)
tot = tot.sort_values(ascending=False)

# TextParser 还有一个 get_chunk 方法，可以读取任意大小的块

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


### 将数据写出到文本格式

In [38]:
data = pd.read_csv('Examples\ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [39]:
data.to_csv('Examples\out.csv')
!cat Examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [42]:
import sys
data.to_csv(sys.stdout,sep = '|') # 仅打印

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [46]:
data.to_csv(sys.stdout, na_rep='NULL') # 缺失值标记

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [ ]:
# 禁用写出行和列标签
data.to_csv(sys.stdout, index=False, header=False)

In [49]:
# 写出一部分列
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [51]:
# Series to_csv
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('examples/tseries.csv')
!cat Examples/tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### 处理分隔符格式

In [52]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [62]:
import csv
f = open('Examples\ex7.csv')
reader = csv.reader(f)
reader

In [56]:
# 对这个reader进行迭代将会为每行产生一个元组（并移除了所有的引号）
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [57]:
with open('examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [58]:
# 标题行和数据行
header, values = lines[0], lines[1:]

In [59]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [63]:
# CSV文件的形式有很多。
# 只需定义csv.Dialect的一个子类即可定义出新格式（如专门的分隔符、字符串引用约定、行结束符等）
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

reader = csv.reader(f, dialect=my_dialect)

可用的选项（csv.Dialect的是属性）及其功能：

![](Img/7178691-7a1cee622459072b.png)

In [66]:
# 手工输出分隔符文件
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

pd.read_csv('mydata.csv',sep=';')

,one,two,three
0,1,2,3
1,4,5,6
2,7,8,9


## Json 数据

JSON（JavaScript Object Notation的简称）已经成为通过HTTP请求在Web浏览器和其他应用程序之间发送数据的标准格式之一。它是一种比表格型文本格式（如CSV）灵活得多的数据格式。

除其空值null和一些其他的细微差别（如列表末尾不允许存在多余的逗号）之外，JSON非常接近于有效的Python代码。基本类型有对象（字典）、数组（列表）、字符串、数值、布尔值以及null。对象中所有的键都必须是字符串。许多Python库都可以读写JSON数据。我将使用json，因为它是构建于Python标准库中的。通过json.loads即可将JSON字符串转换成Python形式。

如何将（一个或一组）JSON对象转换为DataFrame或其他便于分析的数据结构就由你决定了。最简单方便的方式是：向DataFrame构造器传入一个字典的列表（就是原先的JSON对象），并选取数据字段的子集。

pandas.read_json可以自动将特别格式的JSON数据集转换为Series或DataFrame。

In [67]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [69]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [71]:
# json.dumps则将Python对象转换成JSON格式
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

In [72]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


In [73]:
!cat Examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [74]:
data = pd.read_json('examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [75]:
# 从 Pandas 输出到 JSON
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [76]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


## XML和HTML：Web信息收集

Python有许多可以读写常见的HTML和XML格式数据的库，包括lxml、Beautiful Soup和html5lib。lxml的速度比较快，但其它的库处理有误的HTML或XML文件更好。

pandas有一个内置的功能，read_html，它可以使用lxml和Beautiful Soup自动将HTML文件中的表格解析为DataFrame对象。


In [ ]:
!conda install lxml
!pip install beautifulsoup4 html5lib

In [77]:
# pandas.read_html有一些选项，默认条件下，
# 它会搜索、尝试解析<table>标签内的的表格数据。
tables = pd.read_html('Examples\fdic_failed_bank_list.html')

ValueError: No tables found

In [ ]:
len(tables)
failures = tables[0]
failures.head()

In [ ]:
# 数据清洗
close_timestamps = pd.to_datetime(failures['Closing Data'])
close_timestamps.dt.year.value_counts() # 按年份计算倒闭的银行数

### 利用lxml.objectify解析XML

XML（Extensible Markup Language）是另一种常见的支持分层、嵌套数据以及元数据的结构化数据格式。本书所使用的这些文件实际上来自于一个很大的XML文档。

## 二进制数据格式

实现数据的高效二进制格式存储最简单的办法之一是使用Python内置的pickle序列化。pandas对象都有一个用于将数据以pickle格式保存到磁盘上的to_pickle方法。

pandas内置支持两个二进制数据格式：HDF5和MessagePack。


In [78]:
frame = pd.read_csv('Examples\ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [80]:
# 存储
frame.to_pickle('Examples/frame_pickle')

In [81]:
# 读取
pd.read_pickle('examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### HDF5格式

HDF5是一种存储大规模科学数组数据的非常好的文件格式。它可以被作为C标准库，带有许多语言的接口，如Java、Python和MATLAB等。HDF5中的HDF指的是层次型数据格式（hierarchical data format）。每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，HDF5支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写。

In [82]:
frame = pd.DataFrame({'a':np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [84]:
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [85]:
# HDF5文件中的对象可以通过与字典一样的API进行获取
store['obj1']

,a
0,-0.676099
1,-2.620356
2,0.126775
3,-1.671479
4,0.429427
...,...
95,-0.434801
96,1.066347
97,-0.129369
98,0.437652


In [87]:
# HDFStore支持两种存储模式，'fixed'和'table'。
# 后者通常会更慢，但是支持使用特殊语法进行查询操作

store.put('obj2',frame,format='table')
store.select('obj2',where=['index >= 10 and index <= 15'])
store.close()

,a
10,0.102343
11,-0.396793
12,-1.338929
13,-1.204355
14,-0.256767
15,-0.076933


In [89]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5','obj3',where=['index < 5'])

ValueError: The file 'mydata.h5' is already opened, but not in read-only mode (as requested).

如果需要本地处理海量数据，我建议你好好研究一下PyTables和h5py，看看它们能满足你的哪些需求。由于许多数据分析问题都是IO密集型（而不是CPU密集型），利用HDF5这样的工具能显著提升应用程序的效率。

>注意：HDF5不是数据库。它最适合用作“一次写多次读”的数据集。虽然数据可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会被破坏。

## 读取 Microsoft Excel 文件

pandas的ExcelFile类或pandas.read_excel函数支持读取存储在Excel 2003（或更高版本）中的表格型数据。这两个工具分别使用扩展包xlrd和openpyxl读取XLS和XLSX文件。

如果要读取一个文件中的多个表单，创建ExcelFile会更快，但你也可以将文件名传递到pandas.read_excel。

In [90]:
xlsx = pd.ExcelFile('Examples/ex1.xlsx')
xlsx

In [91]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [92]:
frame = pd.read_excel('Examples/ex1.xlsx','Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [94]:
# Pandas 写入 Excel
# 法1：ExcelWriter
writer = pd.ExcelWriter('Examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

In [95]:
# 法2:to_excel
frame.to_excel('Examples/ex2.xlsx')

## Web APIs 交互

许多网站都有一些通过JSON或其他格式提供数据的公共API。通过Python访问这些API的办法有不少。一个简单易用的办法（推荐）是requests包（http://docs.python-requests.org）。

In [ ]:
# 搜索最新的30个GitHub上的pandas主题
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp # <Response [200]>

In [ ]:
data = resp.json()
dataata[0]['title']

In [ ]:
issues = pd.DataFrame(data, columns=['number', 'title','labels',
                                     'states'])
issues

## 数据库交互

在商业场景下，大多数数据可能不是存储在文本或Excel文件中。基于SQL的关系型数据库（如SQL Server、PostgreSQL和MySQL等）使用非常广泛，其它一些数据库也很流行。数据库的选择通常取决于性能、数据完整性以及应用程序的伸缩性需求。

将数据从SQL加载到DataFrame的过程很简单，此外pandas还有一些能够简化该过程的函数。

In [ ]:
import sqlite3
query = '''
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
 );'''
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)

In [ ]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

In [ ]:
cursor.description

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)